In [1]:
import gdal
import ogr
import time

In [2]:
# Computing time
t = time.time()

In [3]:
# Import the catchment(.shp) and get layers, spatial reference and the extent (bounding box)
catch1 = r"C:\Users\Christian Bouman\Documents\2017-2018\Thesis\Data\MOPEX\Catch_11213500\catch_11213500_EPSG3310.shp"
driver = ogr.GetDriverByName('ESRI Shapefile')
catch1 = driver.Open(catch1, 0)
layer = catch1.GetLayer()
spatialRef = layer.GetSpatialRef()
offset = 5000
minX, maxX, minY, maxY = layer.GetExtent()
bbox = ((minX-offset), (maxY+offset), (maxX+offset), (minY-offset))
print(bbox)

(71151.99577297922, -91544.79605918843, 153399.9557027978, -162566.59010958532)


In [4]:
# Open DEM and crop to bbox extent
demfile = r"C:\Users\Christian Bouman\Documents\2017-2018\Thesis\Data\GMTED2010\GMTED2010_EPSG3310.tif"
dem = gdal.Open(demfile)

path_croppeddem = r"C:\Users\Christian Bouman\Documents\2017-2018\Thesis\Data\GMTED2010\GMTED2010_EPSG3310_cropped.tif"
dem_cropped = gdal.Translate(path_croppeddem, dem, projWin=bbox)

In [5]:
# Split DEM in smaller tiles (20x20 pixels)
geotransform = dem_cropped.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]
cols = dem_cropped.RasterXSize
rows = dem_cropped.RasterYSize

npixels = 20
maxY = originY - rows*pixelHeight

x = []
y = []

for i in range(0, cols+1, npixels):
    x0 = originX
    xnew = x0 + i*pixelWidth
    x.append(xnew)

for j in range(0, rows+1, npixels):
    y0 = originY
    ynew = y0 + j*pixelHeight
    y.append(ynew)


for k in range(1, len(y)):
    for l in range(len(x)):
        filename = 'tile_'+ str(k)+", "+str(l)
        path_tiles = r"C:\Users\Christian Bouman\Documents\2017-2018\Thesis\Data\GMTED2010\Tiles\dem_" +str(filename)+".tiff"
        gdal.Translate(path_tiles, dem_cropped, projWin = [x[l-1], y[k-1], x[l], y[k]]) 

In [6]:
#Computing time
elapsed = time.time() - t
print(elapsed)

1.6875383853912354
